## LLM OpenAI Evaluations on LlamaIndex Data

- Faithfulness
- RAG correctness
- Retriever Relevance

In [ ]:
! pip install https://pai-llm-evals.oss-cn-zhangjiakou.aliyuncs.com/sdk/package/pai_llm_evals-0.0.1-py3-none-any.whl

In [ ]:
import configparser
import nest_asyncio
import os
import pandas as pd

from getpass import getpass

from pai.llm_eval.data.data_adapter import LlamaIndexDataAdapter
from pai.llm_eval.data.data_reader import LlamaIndexDataReader
from pai.llm_eval.evals.models.openai_model import OpenAIModel
from pai.llm_eval.evals.default_templates import DefaultPromptTemplateCN
from pai.llm_eval.evals.evaluators import LLMEvaluator
from pai.llm_eval.evals.executors import run_evals

In [ ]:
import pandas as pd
import sys

file_path = (
    "/home/xiaowen/xiaowen/github_code/PAI-RAG/example_data/pai_llm_eval_results.xlsx"
)
csv_file_path = (
    "/home/xiaowen/xiaowen/github_code/PAI-RAG/example_data/pai_llm_eval_results.csv"
)

# 从命令行参数获取文件名
xlsx_file = file_path
csv_file = csv_file_path

# 读取 Excel 文件并转换为 CSV
df = pd.read_excel(xlsx_file)
df.to_csv(csv_file, index=False)

In [ ]:
# read llama index data
file_path = (
    "/home/xiaowen/xiaowen/github_code/PAI-RAG/example_data/pai_llm_eval_results.csv"
)
data = LlamaIndexDataReader.read_data(file_path)

In [ ]:
# select the first two rows of data as the input
eval_input_data = data[:2]
print(f"eval_input_data:\n {eval_input_data}")

eval_input_data:
    Unnamed: 0                             query  \
0           0                 组件化如何简化模型开发并提高效率？   
1           1  如何通过Keras Layer类的组件化设计提高模型开发的效率？   

                                  reference_contexts  \
0  ['为何需要组件化¶\n1. 依靠动态可插拔的公共组件，方便为现有模型添加新特性。¶\n过去...   
1  ['添加新的特性将变得更加容易。¶\n现在我们只需要为新的特征开发一个Keras Layer...   

                                   reference_node_id  \
0  ['182bca8f8d785f74c7173ab47359a663164c408fe857...   
1  ['13bed26b44134f96812e4f758bfa1b00b498c104ac42...   

                                    reference_answer  \
0  组件化通过以下几个方面简化模型开发并提高效率：\n\n1. **动态可插拔性**：组件化允许...   
1  通过Keras Layer类的组件化设计，模型开发的效率得以提高，主要体现在以下几个方面：\...   

                                     response_answer  \
0  组件化通过模块化和配置化的方法简化了模型开发流程。它允许开发者专注于设计独特的子模块（组件）...   
1  通过Keras Layer类的组件化设计，模型开发效率提升体现在能够快速实验和迭代新特性。开...   

                                       retrieved_ids  \
0  ['13bed26b44134f96812e4f758bfa1b00b498c104ac42...   
1  ['182bca8f8d785f74c7173ab47359a663164

In [ ]:
# adapt llama-index data
tda = LlamaIndexDataAdapter()
output_data = tda.convert(eval_input_data)
print(f"output_data:\n {output_data}")

output_data:
 defaultdict(<class 'list'>, {'llm': [{'idx': 0, 'input': '组件化如何简化模型开发并提高效率？', 'output': '组件化通过模块化和配置化的方法简化了模型开发流程。它允许开发者专注于设计独特的子模块（组件），这些组件可以独立开发和测试，然后在需要时轻松地组合和配置，就像拼积木一样。这种结构减少了代码重复，降低了出错风险，特别是对于不熟悉整个框架细节的新手来说，可以直接利用现成的组件，降低了学习曲线，从而提升了整体的开发效率。', 'reference': "['为何需要组件化¶\\n1. 依靠动态可插拔的公共组件，方便为现有模型添加新特性。¶\\n过去一个新开发的公共可选模块，比如Dense Feature Embedding Layer、 SENet添加到现有模型中，需要修改所有模型的代码才能用上新的特性，过程繁琐易出错。随着模型数量和公共模块数量的增加，为所有模型集成所有公共可选模块将产生组合爆炸的不可控局面。组件化实现了底层公共模块与上层模型的解耦。\\n\\n2. 通过重组已有组件，实现“搭积木”式新模型开发。¶\\n很多模型之所以被称之为一个新的模型，是因为引入了一个或多个特殊的子模块（组件），然而这些子模块并不仅仅只能用在该模型中，通过组合各个不同的子模块可以轻易组装一个新的模型。组件化EasyRec支持通过配置化的方式搭建新的模型。\\n\\n3.']"}, {'idx': 1, 'input': '如何通过Keras Layer类的组件化设计提高模型开发的效率？', 'output': '通过Keras Layer类的组件化设计，模型开发效率提升体现在能够快速实验和迭代新特性。开发者只需创建一个专门的Keras Layer类，定义其功能并在相应的package中导入，框架会自动将其纳入组件库。这样，新特性开发简化为开发单一模块，而无需深入理解EasyRec的全部细节。这加快了模型定制和实验的速度，提高了整体开发效率。', 'reference': "['添加新的特性将变得更加容易。¶\\n现在我们只需要为新的特征开发一个Keras Layer类，并在指定package中添加import语句，框架就能自动识别并添加到组件库中，不需要额外操作。开发一个新的模型，只

In [ ]:
retrieval_data = output_data["retrieve"]
llm_data = output_data["llm"]
print(type(llm_data))
llm_data

<class 'list'>


[{'idx': 0,
  'input': '组件化如何简化模型开发并提高效率？',
  'output': '组件化通过模块化和配置化的方法简化了模型开发流程。它允许开发者专注于设计独特的子模块（组件），这些组件可以独立开发和测试，然后在需要时轻松地组合和配置，就像拼积木一样。这种结构减少了代码重复，降低了出错风险，特别是对于不熟悉整个框架细节的新手来说，可以直接利用现成的组件，降低了学习曲线，从而提升了整体的开发效率。',
  'reference': "['为何需要组件化¶\\n1. 依靠动态可插拔的公共组件，方便为现有模型添加新特性。¶\\n过去一个新开发的公共可选模块，比如Dense Feature Embedding Layer、 SENet添加到现有模型中，需要修改所有模型的代码才能用上新的特性，过程繁琐易出错。随着模型数量和公共模块数量的增加，为所有模型集成所有公共可选模块将产生组合爆炸的不可控局面。组件化实现了底层公共模块与上层模型的解耦。\\n\\n2. 通过重组已有组件，实现“搭积木”式新模型开发。¶\\n很多模型之所以被称之为一个新的模型，是因为引入了一个或多个特殊的子模块（组件），然而这些子模块并不仅仅只能用在该模型中，通过组合各个不同的子模块可以轻易组装一个新的模型。组件化EasyRec支持通过配置化的方式搭建新的模型。\\n\\n3.']"},
 {'idx': 1,
  'input': '如何通过Keras Layer类的组件化设计提高模型开发的效率？',
  'output': '通过Keras Layer类的组件化设计，模型开发效率提升体现在能够快速实验和迭代新特性。开发者只需创建一个专门的Keras Layer类，定义其功能并在相应的package中导入，框架会自动将其纳入组件库。这样，新特性开发简化为开发单一模块，而无需深入理解EasyRec的全部细节。这加快了模型定制和实验的速度，提高了整体开发效率。',
  'reference': "['添加新的特性将变得更加容易。¶\\n现在我们只需要为新的特征开发一个Keras Layer类，并在指定package中添加import语句，框架就能自动识别并添加到组件库中，不需要额外操作。开发一个新的模型，只需要实现特殊的新模块，其余部分可以通过组件库中的已有组件拼装。新人不再需要

In [ ]:
import json

qca_dataset = "/home/xiaowen/xiaowen/github_code/PAI-RAG/localdata/eval_exp_data/storage__exp1/predicted_qca_dataset.json"
with open(qca_dataset, "r", encoding="utf-8") as f:
    qcas = json.load(f)
    qcas = qcas["examples"][0:5]

# data = pd.read_json(qcas)
response_df = pd.DataFrame(qcas)
response_df
response_eval_df = response_df[
    ["query", "predicted_contexts", "predicted_answer"]
].rename(
    columns={
        "query": "input",
        "predicted_answer": "output",
        "predicted_contexts": "reference",
    }
)
response_eval_df

,input,reference,output
0,根据你提供的内容，以下是三个适合作为问答对数据集的问题：,[EasyRec是一个易于使用的推荐框架¶\nEasyRec 实现了常见推荐任务中使用的最先...,根据提供的内容，以下是三个适合作为问答对数据集的问题及其答案：\n\n1. **问题**: ...
1,EasyRec 支持在哪些平台上运行？,[EasyRec是一个易于使用的推荐框架¶\nEasyRec 实现了常见推荐任务中使用的最先...,EasyRec 支持在以下平台上运行：\n\n- MaxCompute\n- 数据科学平台\...
2,EasyRec 实现了哪些常见的推荐任务中的机器学习模型？,[EasyRec是一个易于使用的推荐框架¶\nEasyRec 实现了常见推荐任务中使用的最先...,EasyRec 实现了以下常见的推荐任务中的机器学习模型：\n\n- 候选生成（匹配）\n-...
3,EasyRec 如何提高生成高性能模型的效率？,[EasyRec是一个易于使用的推荐框架¶\nEasyRec 实现了常见推荐任务中使用的最先...,EasyRec 提高生成高性能模型的效率主要通过以下几个方面：\n\n1. **简单的配置*...
4,这些问题都基于提供的内容，并且符合要求的具体性、明确性和避免指代不明确的原则。,[为何需要组件化¶\n1. 依靠动态可插拔的公共组件，方便为现有模型添加新特性。¶\n过去一...,根据提供的文档内容，以下是针对相关问题的回答：\n\n1. **为何需要组件化？**\n ...


In [ ]:
retrieval_df = pd.DataFrame(retrieval_data)
response_df = pd.DataFrame(llm_data)
response_df

,idx,input,output,reference
0,0,组件化如何简化模型开发并提高效率？,组件化通过模块化和配置化的方法简化了模型开发流程。它允许开发者专注于设计独特的子模块（组件）...,['为何需要组件化¶\n1. 依靠动态可插拔的公共组件，方便为现有模型添加新特性。¶\n过去...
1,1,如何通过Keras Layer类的组件化设计提高模型开发的效率？,通过Keras Layer类的组件化设计，模型开发效率提升体现在能够快速实验和迭代新特性。开...,['添加新的特性将变得更加容易。¶\n现在我们只需要为新的特征开发一个Keras Layer...


In [ ]:
# check if the openai api key exists in env
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
# Use OpenAI model for evals
eval_model = OpenAIModel(model="gpt-3.5-turbo", api_key=openai_api_key)

In [ ]:
# Create LLM evaluators with eval model and prompt templates
faithfulness_evaluator = LLMEvaluator(
    eval_model, DefaultPromptTemplateCN.FAITHFULNESS_PROMPT_TEMPLATE
)
correctness_evaluator = LLMEvaluator(
    eval_model, DefaultPromptTemplateCN.RAG_CORRECTNESS_PROMPT_TEMPLATE
)
relevance_evaluator = LLMEvaluator(
    eval_model, DefaultPromptTemplateCN.RETRIEVER_RELEVANCE_PROMPT_TEMPLATE
)

In [ ]:
# Run the event loop with nest_asyncio to improve efficiency
nest_asyncio.apply()

In [ ]:
# Evaluate faithfulness and correctness
faithfulness_eval_df, correctness_eval_df = run_evals(
    dataframe=response_eval_df,
    evaluators=[faithfulness_evaluator, correctness_evaluator],
    provide_explanation=True,
)
print(f"faithfulness_eval_df:\n {faithfulness_eval_df}")
print(f"\n\ncorrectness_eval_df:\n {correctness_eval_df}")

faithfulness_eval_df:
   label  score                                        explanation  \
0    事实      1  根据参考文本，答案中提到的EasyRec支持的输入数据类型和提供的模型是事实，与参考文本中的...   
1    事实      1  根据参考文本，EasyRec支持在MaxCompute、数据科学平台、DLC和本地环境上运行...   
2    事实      1  根据参考文本，EasyRec实现了常见推荐任务中的机器学习模型，包括候选生成、评分和多任务学...   
3    事实      1  根据参考文本，答案中提到的EasyRec如何提高生成高性能模型的效率的方法都是基于参考文本中...   
4    事实      1  根据参考文本内容，答案中提到的组件化的优势和EasyRec框架的特点都是基于参考文本中提供的...   

    metric_name eval_type  
0  faithfulness       llm  
1  faithfulness       llm  
2  faithfulness       llm  
3  faithfulness       llm  
4  faithfulness       llm  


correctness_eval_df:
   label  score                                        explanation  \
0   不正确      0  参考文本提到EasyRec支持MaxCompute表、HDFS文件、操作系统文件、卡夫卡流和...   
1    正确      1  参考文本提到EasyRec支持在MaxCompute、数据科学平台、DLC、本地环境上运行，...   
2    正确      1  The answer correctly identifies the machine le...   
3    正确      1  The answer correctly explains how EasyRec impr...   
4    正确      1   根据提供的参考文

In [ ]:
faithfulness_eval_df["score"].tolist()

[1, 1, 1, 1, 1]

In [ ]:
responses = [
    {
        "faithfulness_score": f_s,
        "faithfulness_reason": f_e,
        "correctness_score": c_s,
        "correctness_reason": c_e,
    }
    for f_s, f_e, c_s, c_e in zip(
        faithfulness_eval_df["score"].tolist(),
        faithfulness_eval_df["explanation"].tolist(),
        correctness_eval_df["score"].tolist(),
        correctness_eval_df["explanation"].tolist(),
    )
]
responses

[{'faithfulness_score': 1,
  'faithfulness_reason': '根据参考文本，答案中提到的EasyRec支持的输入数据类型和提供的模型是事实，与参考文本中的信息一致。因此，答案是"事实"。',
  'correctness_score': 0,
  'correctness_reason': '参考文本提到EasyRec支持MaxCompute表、HDFS文件、操作系统文件、卡夫卡流和本地CSV作为输入数据类型，但答案只提到了MaxCompute表、HDFS文件、操作系统文件、Kafka流和本地CSV，漏掉了卡夫卡流一项。因此，答案只回答了部分问题，不完整，应为"不正确"。'},
 {'faithfulness_score': 1,
  'faithfulness_reason': '根据参考文本，EasyRec支持在MaxCompute、数据科学平台、DLC和本地环境上运行。此外，它还支持不同版本的TensorFlow，包括TF1.12-1.15、TF2.x和PAI-TF。因此，答案中提到的平台和TensorFlow版本都是基于参考文本中的信息，没有提供虚假信息。因此，答案是事实。',
  'correctness_score': 1,
  'correctness_reason': '参考文本提到EasyRec支持在MaxCompute、数据科学平台、DLC、本地环境上运行，并且可以在不同的TensorFlow版本上运行。答案中列出了这些平台和TensorFlow版本，说明了EasyRec具有广泛的平台兼容性。因此，答案完整回答了问题并提供了详细信息，可以判断为"正确"。'},
 {'faithfulness_score': 1,
  'faithfulness_reason': '根据参考文本，EasyRec实现了常见推荐任务中的机器学习模型，包括候选生成、评分和多任务学习。参考文本中也提到了具体的模型实例，如DSSM、MIND、DropoutNet等，这些模型并不完全对应于上述三大类推荐任务，而是基于这些基础任务之上构建的具体实现或变种。因此，答案中提到的模型类型和相关信息都可以在参考文本中找到支持，没有提供虚假信息。因此，答案是“事实”。',
  'correctness_score': 1,
  'correct

In [ ]:
# Evaluate retrieval relevance
relevance_eval_df = run_evals(
    dataframe=retrieval_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]
print(f"relevance_eval_df:\n {relevance_eval_df}")

relevance_eval_df:
    idx     label   score                                        explanation  \
0    0  [相关, 相关]  [1, 1]  [参考文本提到组件化简化模型开发并提高效率，描述了如何添加新特性并提高开发效率，因此与问题相...   
1    1  [相关, 相关]  [1, 1]  [参考文本提到通过重组已有组件实现“搭积木”式新模型开发，这与问题中询问如何通过Keras ...   

                                         document_id  \
0  [13bed26b44134f96812e4f758bfa1b00b498c104ac42e...   
1  [182bca8f8d785f74c7173ab47359a663164c408fe857c...   

                               document_score          metric_name eval_type  
0  [0.09999999999999999, 0.09836065573770493]  retriever_relevance  retrieve  
1  [0.09918032786885246, 0.09918032786885246]  retriever_relevance  retrieve  


In [ ]:
kwargs = {
    "messages": [
        {
            "role": "user",
            "content": '\n    在此任务中，您将看到一个查询、参考文本和答案。答案是根据参考文本生成问题。答案可能包含虚假信息，您\n    必须使用参考文本来确定问题的答案是否包含虚假信息，如果答案是事实的幻觉。您的目标是确定参考文本是否\n    包含事实信息，并非幻觉。本文中的“幻觉”是指答案不是基于参考文本或假设的信息不可用参考文本。\n\n        [开始数据]\n        **********\n        [查询]：机器学习PAI是什么？\n        **********\n        [参考文字]：[\'机器学习PAI（Platform of Artificial Intelligence）是阿里云人工智能平台，提供一站式的机器学习解决方案。本文为您介绍什么是机器学习PAI。\\n\\n什么是机器学习\\n机器学习是指机器通过统计学算法，对大量历史数据进行学习，进而利用生成的经验模型指导业务。目前机器学习主要应用在以下场景：\\n营销类场景：商品推荐、用户群体画像或广告精准投放。\\n金融类场景：贷款发放预测、金融风险控制、股票走势预测或黄金价格预测。\\n社交网络服务关系挖掘场景：微博粉丝领袖分析或社交关系链分析。\\n文本类场景：新闻分类、关键词提取、文章摘要或文本内容分析。\\n非结构化数据处理场景：图片分类或图片文本内容提取。\\n其它各类预测场景：降雨预测或足球比赛结果预测。\\n机器学习包括传统机器学习和深度学习。传统机器学习分为以下几类：\\n有监督学习（Supervised Learning）：每个样本都有对应的期望值，通过搭建模型，实现从输入特征向量到目标值的映射。例如解决回归和分类问题。\\n无监督学习（Unsupervised Learning）：所有样本没有目标值，期望从数据本身发现一些潜在规律。例如解决聚类问题。\', \'其它各类预测场景：降雨预测或足球比赛结果预测。\\n机器学习包括传统机器学习和深度学习。传统机器学习分为以下几类：\\n有监督学习（Supervised Learning）：每个样本都有对应的期望值，通过搭建模型，实现从输入特征向量到目标值的映射。例如解决回归和分类问题。\\n无监督学习（Unsupervised Learning）：所有样本没有目标值，期望从数据本身发现一些潜在规律。例如解决聚类问题。\\n增强学习（Reinforcement Learning）：相对比较复杂，系统和外界环境不断交互，根据外界反馈决定自身行为，达到目标最优化。例如阿尔法围棋和无人驾驶。\']\n        **********\n        [答案]：机器学习PAI（Platform of Artificial Intelligence）是阿里云提供的一站式机器学习解决方案。它旨在帮助用户通过使用统计学算法对大量历史数据进行学习，进而利用生成的经验模型来指导业务。机器学习PAI支持多种应用场景，包括但不限于营销类场景、金融类场景、社交网络服务关系挖掘场景、文本类场景以及非结构化数据处理场景等。此外，机器学习PAI涵盖了传统机器学习和深度学习技术，其中传统机器学习又细分为有监督学习、无监督学习和增强学习等多种类型。\n        **********\n        [数据结束]\n\n        根据查询和参考文本，上述答案是事实还是幻觉？\n\n    请仔细阅读查询、参考文本和答案，然后逐步写出说明如何确定答案是“事实”还是“幻觉”。简单地避免\n    一开始就说出正确答案。您的回复标签应该是一个单词：要么“事实”或“幻觉”，并且不应包含任何其他\n    文本或字符。 “产生幻觉”表示答案向基于查询的查询提供了事实上不准确的信息参考文本。 “事实”表\n    明问题的答案相对于事实是正确的参考文本，不包含虚构信息。\n\n    仔细思考一会，然后给出你的结论。你返回的内容是一个json数据结构，其中包含以下字段："label"，"reason"。\n    返回格式如下：{"label":"<标签>", "reason":"<理由>"}\n    ',
        }
    ],
    "model": "qwen-max",
    "temperature": 0.0,
    "max_tokens": 256,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "top_p": 0.99,
    "n": 1,
    "timeout": None,
    "tools": [
        {
            "type": "function",
            "function": {
                "name": "record_response",
                "description": "A function to record your response.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "explanation": {
                            "type": "string",
                            "description": "Explanation of the reasoning for your response.",
                        },
                        "response": {
                            "type": "string",
                            "description": "Your response.",
                            "enum": ["事实", "幻觉"],
                        },
                    },
                    "required": ["explanation", "response"],
                },
            },
        }
    ],
    "tool_choice": {"type": "function", "function": {"name": "record_response"}},
    "response_format": {"type": "json_object"},
}

import openai

_client = openai.OpenAI(
    api_key="sk-499ae44046ee48dfbcb38b6a47045b05",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

res = _client.chat.completions.create(**kwargs).model_dump()

print(res)

{'id': 'chatcmpl-53c849cb-daeb-9c02-9b9d-4e803b0a6cff', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': '', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_6e649ed77cbc4ea295fdf6', 'function': {'arguments': '', 'name': 'record_response\n": {"'}, 'type': 'function', 'index': 0}]}}], 'created': 1729246227, 'model': 'qwen-max', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': None, 'usage': {'completion_tokens': 5, 'prompt_tokens': 1066, 'total_tokens': 1071}}
